In [1]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [2]:
# Create an interactive map
Map = geemap.Map(center=[-23.36, -46.36], zoom=5, add_google_map=True)

Map

Map(center=[-23.36, -46.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [3]:
# Definindo a barra interativa

style = {'description_width': 'initial'}

# Definindo as opções de índices
nd_options =["Índice de Vegetação por Diferença Normalizada (NDVI)", 
              "Índice da Água por Diferença Normalizada (NDWI)",
              "Índice da Água por Diferença Normalizada Melhorado (MNDWI)",
              "Índice do Solo por Diferença Normalizada (NDSI)"]

# Definindo primeira banda
first_band = widgets.Dropdown(
    description='1ª banda:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='Green',
    style=style,
    layout={'width':'150px', 'left':'20px'}
)

# Definindo segunda banda
second_band = widgets.Dropdown(
    description='2ª banda:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='SWIR1',
    style=style,
    layout={'width':'150px', 'left':'40px'}
)

output_widget = widgets.Output(layout={'border': '4px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

aoi_widget = widgets.Checkbox(
    value=True,
    description='Área de interesse',
    style=style
)

download_widget = widgets.Checkbox(
    value=False,
    description='Download dos dados do gráfico',
    style=style
)

def aoi_change(change):
    Map.layers = Map.layers[:4]   
    Map.user_roi = None
    Map.user_rois = None
    Map.draw_count = 0
    output_widget.clear_output()
    
aoi_widget.observe(aoi_change, names='value')

band_combo = widgets.Dropdown(
    description='Band combo:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='NIR/Red/Green',
    style=style
)

year_widget = widgets.IntSlider(min=1984, max=2020, value=2010, description='Selecionar ano:', width=400, style=style, layout={'left':'20px'})

fmask_widget = widgets.Checkbox(
    value=True,
    description='Aplicar fmask?(remove nuvem, sombra e neve)',
    style=style,
    layout = {'width':'2px'}
)


# Normalized Satellite Indices: https://www.usna.edu/Users/oceano/pguth/md_help/html/norm_sat.

nd_indices = widgets.Dropdown(options=nd_options, value=nd_options[0], description='Índices:', style=style, layout = {'width':'450px'})



nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style
)

nd_color = widgets.ColorPicker(
    concise=False,
    description='Color:',
    value='blue',
    style=style
)


def nd_index_change(change):
    if nd_indices.value == 'Índice de Vegetação por Diferença Normalizada (NDVI)':
        first_band.value = 'NIR'
        second_band.value = 'Red'
    elif nd_indices.value == 'Índice da Água por Diferença Normalizada (NDWI)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'Índice da Água por Diferença Normalizada Melhorado (MNDWI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'
    elif nd_indices.value == 'Índice do Solo por Diferença Normalizada (NDSI)':
        first_band.value = 'SWIR1'
        second_band.value = 'NIR'
    elif nd_indices.value == 'Customized':
        first_band.value = None
        second_band.value = None
        
nd_indices.observe(nd_index_change, names='value')

submit = widgets.Button(
    description='Analisar',
    button_style='primary',
    tooltip='Clique aqui',
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([nd_indices, first_band, second_band]),
    widgets.HBox([band_combo, year_widget, fmask_widget]),
    widgets.HBox([aoi_widget, nd_threshold, nd_color, download_widget]),
    submit
])

full_widget

In [4]:
# Click event handler

def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Calculando...')
        Map.default_style = {'cursor': 'wait'}

        try:
            band1 = first_band.value
            band2 = second_band.value
            selected_year = year_widget.value
            threshold = nd_threshold.value
            bands = band_combo.value.split('/')
            apply_fmask = fmask_widget.value
            palette = nd_color.value
            use_aoi = aoi_widget.value
            download = download_widget.value
            
            if use_aoi:
                if Map.user_roi is not None:
                    roi = Map.user_roi
                    layer_name = 'User drawn AOI'
                    geom = roi
                else:
                    output_widget.clear_output() 
                    print('No user AOI could be found.')
                    return

            Map.layers = Map.layers[:4]        
            Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)  
            
            images = geemap.landsat_timeseries(roi=roi, start_year=1984, end_year=2020, start_date='01-01', end_date='12-31', apply_fmask=apply_fmask)
            nd_images = images.map(lambda img: img.normalizedDifference([band1, band2]))
            result_images = nd_images.map(lambda img: img.gt(threshold))

            selected_image = ee.Image(images.toList(images.size()).get(selected_year - 1984))
            selected_result_image = ee.Image(result_images.toList(result_images.size()).get(selected_year - 1984)).selfMask()
            
            vis_params = {
                'bands': bands,
                'min': 0,
                'max': 3000
            }
            
            Map.addLayer(selected_image, vis_params, 'Landsat ' + str(selected_year))
            Map.addLayer(selected_result_image, {'palette': palette}, 'Result ' + str(selected_year))

            
            def cal_area(img):
                pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e4)
                img_area = pixel_area.reduceRegion(**{
                    'geometry': geom,
                    'reducer': ee.Reducer.sum(),
                    'scale': 1000,
                    'maxPixels': 1e12,
                    'bestEffort': True
                })
                return img.set({'area': img_area})
            
            areas = result_images.map(cal_area)
            stats = areas.aggregate_array('area').getInfo()
            x = list(range(1984, 2021))
            y = [item.get('nd') for item in stats]
            
            fig = plt.figure(1)
            fig.layout.height = '270px'
            plt.clear()
            plt.plot(x, y)
            plt.title('Análise temporal (1984-2020)')
            plt.xlabel('Ano')
            plt.ylabel('Area (ha)')
            
            output_widget.clear_output()            

            plt.show()
            
            if download:
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_name = 'chart_' + geemap.random_string() + '.csv'
                out_csv = os.path.join(out_dir, out_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                with open(out_csv, 'w') as f:
                    f.write('ano, area (ha)\n')
                    for index, item in enumerate(x):
                        line = '{},{:.2f}\n'.format(item, y[index])
                        f.write(line) 
                link = geemap.create_download_link(
                    out_csv, title="Clique aqui para download: ")
                display(link)
    
        except Exception as e:
            print(e)
            print('Ocorreu um erro durante os cálculos...')        

        Map.default_style = {'cursor': 'default'}

submit.on_click(submit_clicked)